# **Project Documentation: Web Scraping ETL Pipeline for Cryptocurrency Data**

### **Project Overview**

This project implements a complete Data Acquisition and preprocessing Pipeline using Web scraping. Cryptocurrency market data is collected from the coinmarketcap website then transformed into clean and structured format and then stored in an Sqlite database.
The Pipeline follows an ETL (Extract, Transform and Load) approach and includes both Unit Testing and Integration Testing to show correct  interaction between the data processing layer and the database backend.

In [1]:
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
import sqlite3
import time

/Users/alokekissac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# **Data Source:**

Website: CoinMarketCap

URL: https://coinmarketcap.com  


In [2]:
URL = "https://coinmarketcap.com"

In [3]:
BASE_URL = "https://coinmarketcap.com/"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

COLUMNS = [
    "Name", "Price", "1h %", "24h %", "7d %",
    "Market Cap", "Volume (24h)", "Circulating Supply"
]

df = pd.DataFrame(columns=COLUMNS)


#**Data Acquisition**

Performed using HTTP GET requests through the 'requests' library.
The html content is parsed using BeautifulSoup.

In [4]:
webpage = requests.get(URL, headers=HEADERS)

In [5]:
webpage.status_code

200

In [6]:
type(webpage.content)

bytes

In [7]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/>
  <meta content="https://coinmarketcap.com/" property="og:url"/>
  <link href="https://coinmarketcap.com/" rel="canonical"/>
  <link href="https://coinmarketcap.com/ar/" hreflang="ar" rel="alternate"/>
  <link href="https://coinmarketcap.com/bg/" hreflang="bg" rel="alternate"/>
  <link href="https://coinmarketcap.com/cs/" hreflang="cs" rel="alternate"/>
  <link href="https://coinmarketcap.com/da/" hreflang="da" rel="alternate"/>
  <link href="https://coinmarketcap.com/de/" hreflang="de" rel="alternate"/>
  <link href="https://coinmarketcap.com/el/" hreflang="el" rel="alternate"/>
  <link href="https://coinmarketcap.com/" hreflang="en" rel="alternate"/>
  <link href="https://coinmarketcap.com/es/" hreflang="es" rel="alternate"/>
  <

# **FEATURE EXTRACTION:**

Implemented using a custom rule based extraction function built with BeautifulSoup. The function locates the main cryptocurrency table on the website by searching for a table element with the specific CSS class 'cmc-table'


In [9]:
def tables(soup, df):
    table = soup.find("table", class_="cmc-table") # To find main coinmarketcaptabletable
    if table is None:
        return False

    all_rows = table.find_all("tr")     # Get all table rows

    data_rows = [r for r in all_rows if r.has_attr('style') and 'cursor:pointer' in r['style']]
    if not data_rows:
        data_rows = all_rows[1:]

    for row in data_rows:
        cols = row.find_all("td")
        if len(cols) < 9:
            continue

        values = []
            # NAME
        try:
            name_tag = cols[2].find('p', class_='coin-item-name') or cols[2].find('a')
            name = name_tag.get_text(strip=True) if name_tag else cols[2].get_text(strip=True)
        except Exception:
            name = ''
        values.append(name)

            # PRICE
        try:

            price_tag = cols[3].find('span')
            price = price_tag.get_text(strip=True) if price_tag else cols[3].get_text(strip=True)
        except Exception:
            price = ''
        values.append(price)

            # % CHANGE (1h, 24h, 7d)
        for i in range(4, 7):
            try:
                t = cols[i].find('span')
                val = t.get_text(strip=True) if t else cols[i].get_text(strip=True)
            except Exception:
                val = ''
            values.append(val)

            # MARKET CAP
        try:
            mc_tag = cols[7].find('span') or cols[7]
            market_cap = mc_tag.get_text(strip=True)
        except Exception:
            market_cap = ''
        values.append(market_cap)

            # VOLUME
        try:
            vol_tag = cols[8].find('p') or cols[8].find('span') or cols[8]
            volume = vol_tag.get_text(strip=True)
        except Exception:
            volume = ''
        values.append(volume)

            # CIRCULATING SUPPLY
        try:
            sup_tag = cols[9].find('div') or cols[9]
            supply = sup_tag.get_text(strip=True)
        except Exception:
            supply = ''
        values.append(supply)





        df.loc[len(df)] = values # ENTIRE ROW TO DATAFRAME

    return True


In [10]:
#loop to scrape pages 1 to 6 using tables()function

n_pages = 6

for page in range(1, n_pages + 1):
    print(f"Scraping page {page}...")

    url = f"{BASE_URL}?page={page}" # BUILD URL

    try:
        r = requests.get(url, headers=HEADERS, timeout=15) #DOWNLOAD WEBPAGE
        r.raise_for_status()

        soup = BeautifulSoup(r.text, "lxml")  #HTML INTO BS

        success = tables(soup, df)
        # Using tables() function to extract and append data

        if not success:
            print(f"Warning: No table found on page {page}")

    except Exception as e:
        print(f"Error scraping page {page}: {e}")


    time.sleep(random.uniform(1.0, 2.0))

print("Scraping complete.")
print("Total rows collected:", len(df))




Scraping page 1...
Error scraping page 1: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping page 2...
Error scraping page 2: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping page 3...
Error scraping page 3: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping page 4...
Error scraping page 4: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping page 5...
Error scraping page 5: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping page 6...
Error scraping page 6: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
Scraping complete.
Total rows collected: 0


In [11]:
df

,Name,Price,1h %,24h %,7d %,Market Cap,Volume (24h),Circulating Supply


# **Data Transformation(Preprocessing)**

Cleans and standardises the extracted data which includes the removal of dollar symbols, commas, percentage symbols, alphabetic unit lables and conversion of relevant columns into numeric data types

In [12]:
df["Price"] = (
    df["Price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)

for col in ["1h %", "24h %", "7d %"]:
    df[col] = df[col].str.replace("%", "", regex=False)

for col in ["Market Cap", "Volume (24h)"]:
    df[col]= (
        df[col]
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace(r"[A-Za-z]", "", regex=True)
    )

df["Circulating Supply"] = df["Circulating Supply"].str.replace(
    r"[A-Za-z]", "", regex=True
)

numeric_cols = ["Price", "1h %", "24h %", "7d %", "Market Cap", "Volume (24h)", "Circulating Supply"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

pd.options.display.float_format = '{:,.3f}'.format

In [13]:
df

,Name,Price,1h %,24h %,7d %,Market Cap,Volume (24h),Circulating Supply


In [14]:
df.to_csv("coinmarketcap_data.csv")

# **Data Loading**
After preprocessing the cleaned dataset is stored in an SQLite database using the 'SQLite3'.

In [15]:
conn = sqlite3.connect("crypto_data.db")
df.to_sql("coins", conn, if_exists="replace", index=False)
conn.close()

# **Testing**



*   Unit Testing
*   Integration testing



In [16]:
# UNIT TEST: Price Cleaning

test_df = pd.DataFrame({
    "Price": ["$92,273.82", "$3,238.58", "$1.00"]
})

test_df["Price"] =(
    test_df["Price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)

test_df["Price"] = pd.to_numeric(test_df["Price"], errors="coerce")

assert test_df["Price"].iloc[0] == 92273.82
assert test_df["Price"].iloc[1] == 3238.58
assert test_df["Price"].iloc[2] == 1.00

print("All Tests Passed")

All Tests Passed


In [17]:
conn = sqlite3.connect("crypto_data.db")
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM coins")
row_count = cursor.fetchone()[0]
conn.close()

assert row_count > 0
print("Integration Test Passed")

AssertionError: 

Integration Test passed
Data successfully stored and retrieved from SQlite

# **Academic Integrity & Attribution**

All code in this notebook was written and understood by me as part of this project. The implementation uses commonly available pythonlibraries such as BeautifulSoup, Requests, Pandas, SQLite3, and unittest, all of which are open source and used with their official documentation and licences.

I used Chatgpt only as a guidance during development, to clarify concepts, improve pagination logic, resolve minor errors, assist with testing stratergies, and support the database loading

All integrated libraries comply with their original licences